## Start Up File startup.py

In [2]:
import sqlite3

def create_cnn(file_name):
    # create database file by specifying location
    cnn = sqlite3.connect(file_name)

    return cnn

def create_tables(cnn):
    # initialize cursor object to interact with database
    cur = cnn.cursor()

    # create all necessary tables
    cur.execute("""CREATE TABLE recipe 
                (recipe_id INTEGER PRIMARY KEY NOT NULL, 
                recipe_name TEXT, 
                recipe_notes TEXT,
                difficulty_id INTEGER,
                cuisine_id INTEGER,
                course_id INTEGER,
                diet_id INTEGER,
                instructions_id INTEGER,
                FOREIGN KEY (difficulty_id) REFERENCES difficulty(difficulty_id),
                FOREIGN KEY (cuisine_id) REFERENCES difficulty(cuisine_id),
                FOREIGN KEY (course_id) REFERENCES difficulty(course_id),
                FOREIGN KEY (diet_id) REFERENCES difficulty(diet_id),
                FOREIGN KEY (instructions_id) REFERENCES difficulty(instructions_id))""")

    cur.execute("""CREATE TABLE difficulty
                (difficulty_id INTEGER PRIMARY KEY NOT NULL,
                difficulty TEXT)""")    

    cur.execute("""CREATE TABLE cuisine
                (cuisine_id INTEGER PRIMARY KEY NOT NULL,
                cuisine TEXT)""")

    cur.execute("""CREATE TABLE course
                (course_id INTEGER PRIMARY KEY NOT NULL,
                course TEXT)""")

    cur.execute("""CREATE TABLE diet
                (diet_id INTEGER PRIMARY KEY NOT NULL,
                diet TEXT)""")

    cur.execute("""CREATE TABLE instructions
                (instruction_id INTEGER PRIMARY KEY NOT NULL,
                instructions TEXT)""")

    cur.execute("""CREATE TABLE ingredient
                (ingredient_id INTEGER PRIMARY KEY NOT NULL,
                ingredient TEXT)""")

    cur.execute("""CREATE TABLE unit
                (unit_id INTEGER PRIMARY KEY NOT NULL,
                unit TEXT)""")

    cur.execute("""CREATE TABLE quantity
                (quantity_id INTEGER PRIMARY KEY NOT NULL,
                quantity TEXT)""")

    cur.execute("""CREATE TABLE prepmethod
                (prepmethod_id INTEGER PRIMARY KEY NOT NULL,
                prepmethod TEXT)""")

    cur.execute("""CREATE TABLE recipe_ingredient
                (recipe_id INTEGER,
                ingredient_id INTEGER,
                unit_id INTEGER,
                quantity_id INTEGER,
                prepmethod_id INTEGER,
                FOREIGN KEY (recipe_id) REFERENCES recipe(recipe_id),
                FOREIGN KEY (ingredient_id) REFERENCES ingredient(ingredient_id),
                FOREIGN KEY (unit_id) REFERENCES unit(unit_id),
                FOREIGN KEY (quantity_id) REFERENCES quantity(quantity_id),
                FOREIGN KEY (prepmethod_id) REFERENCES prepmethod(prepmethod_id))""")

    cnn.commit()

def insert_known_table_values(cnn):
    cur = cnn.cursor()

    cur.execute("""INSERT INTO difficulty (difficulty) VALUES ("easy"), ("average"), ("difficult")""")
    cur.execute("""INSERT INTO course (course) VALUES ("appetizer"), ("main"), ("dessert"), ("drinks"), ("soups / stews")""")
    cur.execute("""INSERT INTO diet (diet) VALUES ("None"), ("vegan"), ("vegetarian"), ("gluten-free")""")

    cnn.commit()

## Recipe file recipe.py ##

In [3]:
class Recipe:
    def __init__(self, recipe_name, recipe_notes, ingredients_dict, difficulty, cuisine, course, diet, instructions):
        self.recipe_name = recipe_name
        self.recipe_notes = recipe_notes
        self.ingredients_dict = ingredients_dict #keys: ingredient, unit, quantity, prep method
        self.difficulty = difficulty
        self.cuisine = cuisine
        self.course = course
        self.diet = diet
        self.instructions = instructions
        
    #################
    # inserting recipes
    #################

    def insert_cuisine(self, cnn):
        cur = cnn.cursor()

        sql = "INSERT INTO cuisine (cuisine) SELECT (?) WHERE NOT EXISTS (SELECT 1 FROM cuisine WHERE cuisine = ?)"
        values = (self.cuisine, self.cuisine)
        
        cur.execute(sql, values)
        cnn.commit()
        
    def insert_instructions(self, cnn):
        cur = cnn.cursor()

        sql = "INSERT INTO instructions (instructions) VALUES (?)"
        values = (self.instructions,)
        
        cur.execute(sql, values)
        cnn.commit()
        
    def insert_recipe(self, cnn):
        cur = cnn.cursor()

        sql = """INSERT INTO recipe(recipe_name, recipe_notes, 
                    difficulty_id, cuisine_id, course_id, diet_id, instructions_id)
                    VALUES (?, ?, 
                        (SELECT difficulty_id FROM difficulty WHERE difficulty = ?), 
                        (SELECT cuisine_id FROM cuisine WHERE cuisine = ?),
                        (SELECT course_id FROM course WHERE course = ?),
                        (SELECT diet_id FROM diet WHERE diet = ?), 
                        (SELECT instruction_id FROM instructions WHERE instructions = ?))"""
        
        values = (self.recipe_name, self.recipe_notes, self.difficulty, self.cuisine, self.course, self.diet, self.instructions)

        cur.execute(sql, values)
        cnn.commit()
        
    # ingredients_dict is dictionary with keys: ingredient, unit, quantity, prepmethod
    def insert_ingredient_attr(self, attr, cnn):
        cur = cnn.cursor()

        sql = "INSERT INTO {key} ({key}) SELECT (?) WHERE NOT EXISTS (SELECT 1 FROM {key} WHERE {key} = ?)".format(key = attr)
        
        values = []
        for x in self.ingredients_dict[attr]: 
            values.append((x, x))
            
        cur.executemany(sql, values)
        cnn.commit()
        
    def insert_recipe_ingredient(self, cnn):
        cur = cnn.cursor()

        sql = """INSERT INTO recipe_ingredient (recipe_id, ingredient_id, unit_id, quantity_id, prepmethod_id) VALUES
                ((SELECT recipe_id FROM recipe WHERE recipe_name = ?),
                (SELECT ingredient_id FROM ingredient WHERE ingredient = ?),
                (SELECT unit_id FROM unit WHERE unit = ?),
                (SELECT quantity_id FROM quantity WHERE quantity = ?),
                (SELECT prepmethod_id FROM prepmethod WHERE prepmethod = ?))"""
        
        count = 0
        temp = [self.recipe_name]
        values = []
        while count < len(self.ingredients_dict['ingredient']):
            for key in self.ingredients_dict:
                temp.append(self.ingredients_dict[key][count])
            values.append(tuple(temp))
            temp = [self.recipe_name]
            count = count + 1
        
        cur.executemany(sql, values) #values is a list of tuples
        cnn.commit()
        
    def insert_all(self, cnn):
        self.insert_cuisine(cnn)
        self.insert_instructions(cnn)
        self.insert_recipe(cnn)

        for key in self.ingredients_dict:
            self.insert_ingredient_attr(key, cnn)
        
        self.insert_recipe_ingredient(cnn)


## View file view.py ##

In [236]:
import pandas as pd
    
def print_recipes(cnn, **kwargs):
    """Use a SQL SELECT statement to retrieve recipes based on selections in **kwargs"""

    sql = """SELECT recipe.recipe_id, recipe.recipe_name, recipe.recipe_notes,
                ingredient.ingredient, unit.unit, quantity.quantity, prepmethod.prepmethod,
                instructions.instructions, difficulty.difficulty, cuisine.cuisine,
                course.course, diet.diet

                FROM recipe_ingredient

                INNER JOIN recipe ON recipe.recipe_id = recipe_ingredient.recipe_id
                INNER JOIN ingredient ON ingredient.ingredient_id = recipe_ingredient.ingredient_id
                INNER JOIN unit ON unit.unit_id = recipe_ingredient.unit_id
                INNER JOIN quantity ON quantity.quantity_id = recipe_ingredient.quantity_id
                INNER JOIN prepmethod ON prepmethod.prepmethod_id = recipe_ingredient.prepmethod_id

                INNER JOIN instructions ON instructions.instruction_id = recipe.instructions_id
                INNER JOIN difficulty ON difficulty.difficulty_id = recipe.difficulty_id
                INNER JOIN cuisine ON cuisine.cuisine_id = recipe.cuisine_id
                INNER JOIN course ON course.course_id = recipe.course_id
                INNER JOIN diet ON diet.diet_id = recipe.diet_id
                """
    
    if len(kwargs) == 1: # if there is only one search key, sql string is different
        values = kwargs.popitem()
        if isinstance(values[1], str): # if search parameter is a string, need to format as such in the sql query
            sql2 = "WHERE {column} = '{selection}'".format(column=values[0], selection=values[1].lstrip())
        else:
            sql2 = "WHERE {column} = {selection}".format(column=values[0], selection=values[1])
        sql = sql + sql2

    count = 0
    if len(kwargs) > 1:
        li = []
        # iterate through all the search parameters in kwargs, format search query accordingly 
        for key, value in kwargs.items():
            if isinstance(value, str): 
                string = "{column} = '{selection}'".format(column = key, selection = value.lstrip())
            else:
                string = "{column} = {selection}".format(column = key, selection = value)
            li.append(string)
        count = 0
        concatstring = ""
        while count < len(li) - 1:
            concatstring = concatstring + li[count] + " OR " #concatenate all the search requirements with an "OR"
            count = count + 1

        concatstring = concatstring + li[len(li) -1]

        sql2 = "WHERE " + concatstring

        sql = sql + sql2

    recipe_df = pd.read_sql_query(sql, cnn)

    return recipe_df
        
def convert(recipes_df, conversion_unit):
    # each measurement's equivalent of one unit in US fl oz 
    conversion_chart = {'tsp': 1/6,
                       'tbsp': 1/2,
                       'oz': 1,
                       'cup': 8,
                       'pint': 16,
                       'quart': 32,
                       'gallon': 128}

    to_convert = {}
    converted = {}

    #find if there is more than one of same ingredient in the chosen recipes
    ingr_count_series = recipes_df.groupby('ingredient').nunique()["recipe_id"]

    # if only one instance of ingredient, enter in final dictionary
    for ingr in ingr_count_series[ingr_count_series <= 1].iteritems():
        df = (recipes_df[recipes_df['ingredient'] == ingr[0]])
        converted.update({ingr[0] + " (" + df['unit'].iloc[0] + ")": df['quantity'].iloc[0]})
    
    # if more than one instance of ingredient, need to do some converting
    ingr_li = []
    for ingr in ingr_count_series[ingr_count_series > 1].iteritems():
        ingr_li.append(ingr)
    
    for ingr in ingr_li: # ingr is a tuple of (ingr, count)
        one_ingr_df = recipes_df[recipes_df['ingredient']==ingr[0]] # create df filtered one ingr at a time
        total_floz = 0
        for unit in one_ingr_df['unit'].unique(): # go through all unique units for current ingredient
            if unit in conversion_chart: # if it's a unit that can be converted
                unit_sum = pd.to_numeric(one_ingr_df[one_ingr_df['unit'] == unit]['quantity']).sum()
                floz = unit_sum * conversion_chart[unit]
                total_floz = total_floz + floz # get total amount in fl oz
                to_convert.update({ingr[0] : total_floz})
            else: # if a unit that can't be converted
                unit_sum = pd.to_numeric(one_ingr_df[one_ingr_df['unit'] == unit]['quantity']).sum() # sum quantities of same units
                converted.update({"summed " + ingr[0] + " (" + unit + ")": unit_sum})

    for ingr in to_convert:
        converted.update({("summed " + ingr + " (" + conversion_unit + ")") : to_convert[ingr] / conversion_chart[conversion_unit]})

    return (converted)
    
def ask_convert(recipes_df):
    while True:
        answer = input("Would you like to aggregate your ingredients? Enter 'yes' or 'no'.")
        if answer not in ('yes', 'no'):
            print("Please enter a valid answer.")
            continue
        else:
            break

    if answer == 'no':
        print("See ingredients in table of recipes.")
    else: #if answer == 'yes'
        while True:
            conversion_unit = input("Please choose a unit for your aggregated ingredients (tsp, tbsp, oz, cup, pint, quart, gallon).")
            if conversion_unit not in ('tsp', 'tbsp', 'oz', 'cup', 'pint', 'quart', 'gallon'):
                print("Please enter a valid answer.")
                continue
            else:
                break
        return convert(recipes_df, conversion_unit)

## User Interface Utils  uiutils.py ##

In [242]:
def get_user_input():
    while True:
        answer = input("Do you want to enter or view recipes? Type 'enter' or 'view'. ")
        if answer.lower() not in ('enter', 'view'):
            print("Please type 'enter' or 'view'. ")
            continue
        else:
            break
    return answer.lower()
    
def get_recipe_name():
    recipe_name = input("Enter recipe name: ")
    return recipe_name

def get_recipe_notes():
    recipe_notes = input("Enter recipe notes: ")
    return recipe_notes

def get_recipe_difficulty():
    while True:
        recipe_difficulty = input("Enter recipe difficulty (easy, average, difficult): ")
        if recipe_difficulty not in ('easy', 'average', 'difficult'): 
            print("Please enter a valid difficulty.")
            continue
        else:
            break
    return recipe_difficulty

def get_recipe_cuisine():
    recipe_cuisine = input("Enter recipe cuisine: ")
    return recipe_cuisine

def get_recipe_course():
    while True:
        recipe_course = input("Enter recipe course (appetizer, main, dessert, drinks, soups / stews): ")
        if recipe_course not in ('appetizer', 'main', 'dessert', 'drinks', 'soups / stews'): 
            print("Please enter a valid course.")
            continue
        else:
            break
    return recipe_course

def get_recipe_diet():
    while True:
        recipe_diet = input("Enter recipe diet (None, vegan, vegetarian, gluten-free): ")
        if recipe_diet not in ('None', 'vegan', 'vegetarian', 'gluten-free'): 
            print("Please enter a valid difficulty.")
            continue
        else:
            break
            
    return recipe_diet
        
def get_recipe_ingredient():
    recipe_ingredient_li = []
    recipe_ingredient = {'ingredient': [], 'unit': [], 'quantity': [], 'prepmethod': []}
    while True:
        ingredient = input("""Enter one ingredient at a time with comma separations: ingredient name, unit, quantity, prep method. \nIf attribute does not exist, enter as 'None'. To quit, enter 'q'. """)
        if ingredient == 'q':
            break
        elif len(list(ingredient.split(","))) != 4:
            print("Please enter the four attributes: ingredient name, unit, quantity, and preparation method. Or enter 'q' to quit. ")
            continue
        else:     
            recipe_ingredient_li.append(list(ingredient.split(","))) 
            continue
      
    for entry in recipe_ingredient_li:
        
        recipe_ingredient['ingredient'].append(entry[0].lstrip()) # strip leading white spaces for aesthetic purposes
        recipe_ingredient['unit'].append(entry[1].lstrip())
        recipe_ingredient['quantity'].append(entry[2].lstrip())
        recipe_ingredient['prepmethod'].append(entry[3].lstrip())            
    
    return recipe_ingredient

def get_recipe_instructions():
    recipe_instructions = input("Enter recipe instructions: ")
    return recipe_instructions

def get_search_criteria():
    criteria_li = []
    criteria_dict = {}
    while True:
        criteria = input("Enter each search criteria by [category, value] (e.g. ingredient, fish). Enter 'q' to quit. ")
        if criteria == 'q':
            break
        elif len(list(criteria.split(','))) != 2:
            print("Please enter one category with one value. ")
            continue
        else:
            criteria_li.append(list(criteria.split(',')))
            continue

    for entry in criteria_li:
        criteria_dict[entry[0]] = entry[1]
 
    return criteria_dict
    

## Main file main.py ##

In [241]:
def main():
    cnn = create_cnn(r"C:\Users\jenni\Documents\sqlite\db\mealpreprecipes3.db")

    with cnn:
        # if first starting out, otherwise remove
#         create_tables(cnn)
#         insert_known_table_values(cnn)
        
        # enter option menu
        ans = get_user_input()
        
        #### If user wants to enter a recipe ####
        if ans == 'enter':        
            recipe_name = get_recipe_name()
            recipe_notes = get_recipe_notes()
            difficulty = get_recipe_difficulty()
            cuisine = get_recipe_cuisine()
            course = get_recipe_course()
            diet = get_recipe_diet()
            ingredients_dict = get_recipe_ingredient()
            instructions = get_recipe_instructions()

            recipe = Recipe(recipe_name, recipe_notes, ingredients_dict, difficulty, cuisine, course, diet, instructions)

            recipe.insert_all(cnn)
        
        #### If user wants to view recipes ####
        if ans == 'view':
            criteria_dict = get_search_criteria()
            recipes_df = print_recipes(cnn, **criteria_dict) #TODO: INSERT GET_SEARCH_CRITERIa
            
            print(ask_convert(recipes_df))
            
            return recipes_df

if __name__ == '__main__':
     main()